In [0]:
import pandas as pd

#Create sample sales data
sales_data = {
    "OrderID": [1, 2, 3, 4],
    "OrderDate": ["2024-01-01 10:00:00", "2024-01-02 11:00:00", "2024-01-03 12:00:00", "2024-01-04 13:00:00"],
    "CustomerID": ["C001", "С002", "сеез", "C004"],
    "Product": ["ProductA", "ProductB", "ProductC", "ProductD"], 
    "Quantity": [10, 20, 15, 5],
    "Price": [100.0, 200.0, 150.0, 50.0]
}

#Convert to DataFrame
df_sales = pd.DataFrame(sales_data)

#Save as CSV
csv_path = "/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data.csv"
df_sales.to_csv(csv_path, index=False)

#Save as Parquet
parquet_path = "/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data.parquet" 
df_sales.to_parquet(parquet_path, index=False)

print(f"Sample data saved to {csv_path} and {parquet_path}")

---------------------------------------------------------------------------
OSError                                   Traceback (most recent call last)
File <command-750668376192463>:18
     16 #Save as CSV
     17 csv_path = "/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data.csv"
---> 18 df_sales.to_csv(csv_path, index=False)
     20 #Save as Parquet
     21 parquet_path = "/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data.parquet" 

File /databricks/python/lib/python3.9/site-packages/pandas/core/generic.py:3551, in NDFrame.to_csv(self, path_or_buf, sep, na_rep, float_format, columns, header, index, index_label, mode, encoding, compression, quoting, quotechar, line_terminator, chunksize, date_format, doublequote, escapechar, decimal, errors, storage_options)
   3540 df = self if isinstance(self, ABCDataFrame) else self.to_frame()
   3542 formatter = DataFrameFormatter(
   3543     frame=df,
   3544     header=header,
   (...)
   3548     decimal=decimal,
   354

In [0]:
 # Initialize SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

spark = SparkSession.builder\
    .appName("DeltaExample") \
    .getOrCreate()

# Load data from CSV
df_sales = spark.read.format("csv").option("header", "true").load("/FileStore/sales_data.csv")

# Transform the data: Add a new column for total amount
df_transformed = df_sales.withColumn("TotalAmount", col("Quantity").cast("int") * col("Price").cast("double"))

# Write transformed data to Delta table
delta_table_path = "/delta/sales_data"
df_transformed.write.format("delta").mode("overwrite").save(delta_table_path)

print("Delta table created and data written successfully.")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-750668376192464>:10
      5 spark = SparkSession.builder\
      6    .appName("DeltaExample") \
      7    .getOrCreate()
      9 # Load data from CSV
---> 10 df_sales = spark.read.format("csv").option("header", "true").load("/FileStore/sales_data.csv")
     12 # Transform the data: Add a new column for total amount
     13 df_transformed = df_sales.withColumn("TotalAmount", col("Quantity").cast("int") * col("Price").cast("double"))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/

In [0]:
import dlt

@dlt.table
def sales_data():
    df = spark.read.format("delta").load(delta_table_path)
    return df.select(
        col("OrderID"),
        col ("OrderDate"),
        col("CustomerID"),
        col ("Product"),
        col("Quantity"),
        col("Price"),
        (col("Quantity").cast("int") * col("Price").cast("double")).alias ("TotalAmount")
)

print("Delta Live Table created.")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-750668376192465>:1
----> 1 import dlt
      3 @dlt.table
      4 def sales_data():
      5     df = spark.read.format("delta").load(delta_table_path)

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._nest_level == 1
    174     # `level` represents the number of leading dots in a relative import statement.
    175     # If it's zero, then this

In [0]:
print("hi")

hi


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("DeltaOperationsSimpleExample") \
    .getOrCreate()

# Define Delta table path
delta_table_path = "/delta/simple_data"

# Define initial sample data
initial_data = [
    (1, 100),
    (2, 200),
    (3, 300)
]

# Define schema
schema = ["ID", "Value"]

# Create DataFrame for initial data
df_initial = spark.createDataFrame(initial_data, schema=schema)

# Write DataFrame to Delta table
df_initial.write.format("delta").mode("overwrite").save(delta_table_path)

print("Initial Delta table created and data written successfully.")

# Define new sample data
new_sample_data = [
    (2, 250),  # Existing ID with updated Value
    (4, 400)   # New ID
]

# Create DataFrame for new data
df_new = spark.createDataFrame(new_sample_data, schema=schema)

# Write the new data to Delta table in append mode
df_new.write.format("delta").mode("append").save(delta_table_path)

print("New data appended to Delta table successfully.")

# Create a temporary view for SQL operations
df_new.createOrReplaceTempView("new_data")

# Perform the merge operation
print("Merging new data into Delta table...")

spark.sql(f"""
MERGE INTO delta.`{delta_table_path}` AS target
USING new_data AS source
ON target.ID = source.ID
WHEN MATCHED THEN UPDATE SET
    target.Value = source.Value
WHEN NOT MATCHED THEN INSERT (
    ID,
    Value
) VALUES (
    source.ID,
    source.Value
)
""")

print("Data merged successfully.")

# Delta operations - History, Time Travel, and Vacuum
print("Viewing Delta table history...")
history_df = spark.sql(f"DESCRIBE HISTORY delta.`{delta_table_path}`")
history_df.show(truncate=False)

print("Querying Delta table as of version 0...")
df_time_travel = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
df_time_travel.show(truncate=False)

print("Vacuuming old files...")
spark.sql(f"VACUUM delta.`{delta_table_path}` RETAIN 168 HOURS")

print("Delta operations completed.")


Initial Delta table created and data written successfully.
New data appended to Delta table successfully.
Merging new data into Delta table...
Data merged successfully.
Viewing Delta table history...
+-------+-------------------+----------------+------------------------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------